In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
from dataclasses import dataclass, fields
import sys
from sim import TSim
from profiles.example_basic_queueing.scenario import Args
from lib.arg import Int, Float
from typing import Tuple

class InvalidArgDefinition(Exception):
    pass

def get_bounds(v)->Tuple[float, float]:
    try:
        bounds = v.bounds
        assert len(bounds) ==2
        assert bounds[0] is not None
        assert bounds[1] is not None    
        
        return bounds
    except:
        raise InvalidArgDefinition()

def generate_widgets_for_args_class(dataclass_type):
    """
    Generate a dictionary of ipywidgets based on the params of a dataclass.
    """
    widgets_dict = {}
    style = {'description_width': 'initial'}
    for field in fields(dataclass_type):    
        field_type = field.type       
        descr=field_type.kwargs.get('help', field.name)
        if not len(descr):
            descr = field.name
        wbuild_args = dict(description=descr, style=style)
        try:
            if isinstance(field_type, Int):                
                bounds = get_bounds(field_type)                            
                widgets_dict[field.name] = widgets.IntSlider(value=field.default, min=bounds[0], max=bounds[1], **wbuild_args)
            elif isinstance(field_type, Float):
                bounds = get_bounds(field_type)
                widgets_dict[field.name] = widgets.FloatSlider(value=field.default, min=bounds[0], max=bounds[1], **wbuild_args)
            else:
                raise InvalidArgDefinition()
        except InvalidArgDefinition:
            widgets_dict[field.name] = widgets.Text(value=str(field.default), **wbuild_args)
    return widgets_dict

widgets_dict = generate_widgets_for_args_class(Args)




for widget in widgets_dict.values():
    display(widget)


output_widget = widgets.Output()
results_widget = widgets.Textarea()


def run_simulation(btn):   
    dataclass_type = Args
    kv = {}
    for field in fields(dataclass_type):    
        v = widgets_dict[field.name].value
        print(type(v))
        kv[field.name] = v
        
    args = Args(**kv)
    
    results_stream = io.StringIO("")
    
    with output_widget:        
        clear_output(wait=True) 
        # print(type(results_stream))
        try:
            sim = TSim(stats_storage=results_stream)            
            print("sim created")
            print("Load scenario")
            
            sim.load_scenario("")           
            print("sim start")
            sim.start()
            print("sim end")            
            results_widget.value = results_stream.getvalue()
        except Exception as e:
            from gl_vars import gl
            gl.reset()
            raise
            
        


button = widgets.Button(description="Run scenario")
button.on_click(run_simulation)
display(button, output_widget, results_widget)


Text(value='100', description='PRNG seed', style=TextStyle(description_width='initial'))

Text(value='500', description='duration of the simulation in ticks', style=TextStyle(description_width='initia…

IntSlider(value=2, description='start_time_1', max=42, style=SliderStyle(description_width='initial'))

IntSlider(value=4, description='Start time for flow 2', max=42, style=SliderStyle(description_width='initial')…

Text(value='6', description='start_time_3', style=TextStyle(description_width='initial'))

...

Text(value='15', description='arrival interval for the second queue', style=TextStyle(description_width='initi…

Text(value='20', description='arrival interval for the third queue', style=TextStyle(description_width='initia…

Button(description='Run scenario', style=ButtonStyle())

Output()

Textarea(value='')